# Tutorial for DeepAugment

- Github: https://github.com/barisozmen/deepaugment
- Slides: https://bit.ly/deepaugmentslides
- PyPI: https://pypi.org/project/deepaugment/


In this tutorial, we will:
1. Install
2. Import
3. Setup and configure
4. Optimizer augmentations
5. Print found augmentation policies
6. Create image-generator using augmentation policies
7. Use the image-generator on a full-model

**Important Note:** This notebook is only for showing usage of the library. Configurations are made as running the notebook will take 10-20 minutes (using TPU). Therefore this is not a proper analysis with useful results. A proper analysis would take 5-10 hours. For a proper analysis, change configurations as following:
* **child_epochs:** +100 (Section 3)
* **optimizer iterations:** +100 (Section 4)
* **model:** wrn_28_10 or InceptionV3 (Section 7)

See [slides](http://bit.ly/deepaugmentslides) for results with CIFAR-10 and iNaturalist images, where DeepAugment was ran for +4 hours on AWS p3.x2large instance

### Contact
Baris Ozmen, hbaristr@gmail.com

## 1. Install

### Installed version

In [1]:
!pip freeze | grep deepaugment

## 2.Import

In [2]:
from deepaugment.deepaugment import DeepAugment

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-04-02 14:45:02.652448: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-02 14:45:02.652572: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-02 14:45:03.852128: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-02 14:45:03.852169: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 3. A simple setup of DeepAugment

Number of child epochs are set as 10 in order to make a quick run. For a thorough analysis, set `child_epochs` >=50

In [3]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# child_epochs set to 10 for a quick run, but it should be >=50 for a proper analysis
my_config = {
    "child_epochs":2,
    "opt_samples":1
}

# X_train.shape -> (N, M, M, 3)
# y_train.shape -> (N)
deepaug = DeepAugment(images=x_train, labels=y_train, config=my_config)

Using 2000 training images


2022-04-02 14:45:05.138437: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-02 14:45:05.138503: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


BasicCNN model built as child model.
 Model summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
   

2022-04-02 14:45:08.887786: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-02 14:45:09.462575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-02 14:45:16.695359: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 - 8s - loss: 1.9697 - accuracy: 0.2582 - val_loss: 2.0705 - val_accuracy: 0.2660 - 8s/epoch - 43ms/step
Epoch 2/2
188/188 - 5s - loss: 1.6448 - accuracy: 0.3946 - val_loss: 1.8569 - val_accuracy: 0.3280 - 5s/epoch - 29ms/step
fit()'s runtime:  13.7324 sec.
0, 0.7029999941587448, ['rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0, 'rotate', 0.0]


## 4. Optimize for 1 iteration (normally 100-300 iterations needed)
- Each iteration takes ~30 secs on AWS p3.x2large (V100 GPU), therefore a proper optimization with 300 iterations would take ~2.5 hours and cost ~8$.

    - AWS instance p3.x2large cost: 3.09$/h  

In [7]:
# number of iterations set to 2 (set it >=100 for a proper analysis)
best_policies = deepaug.optimize(2)
#x_portion_aug_max is 305

trial: 1 
 ['dropout', 0.8009107519796445, 'coarse-dropout', 0.6788795301189604, 'fog', 0.5820197920751072, 'coarse-dropout', 0.7586156243223574, 'rotate', 0.47360041934665753, 'shear', 0.7369181771289582, 'translate-x', 0.13521817340545209, 'sharpen', 0.1496748671836832, 'translate-x', 0.38648898112586205, 'horizontal-flip', 0.4499499899112277]
Länge Hyperparams:20
Iteration is:0
X_portion:
[[[[245. 245. 245.]
   [248. 248. 248.]
   [249. 249. 249.]
   ...
   [159. 172. 178.]
   [157. 169. 176.]
   [156. 168. 174.]]

  [[243. 243. 243.]
   [246. 246. 246.]
   [247. 247. 247.]
   ...
   [163. 173. 178.]
   [159. 169. 174.]
   [157. 167. 172.]]

  [[245. 245. 245.]
   [245. 245. 245.]
   [246. 246. 246.]
   ...
   [173. 182. 185.]
   [167. 177. 179.]
   [164. 173. 176.]]

  ...

  [[168. 170. 169.]
   [179. 181. 180.]
   [181. 183. 182.]
   ...
   [ 94. 101. 104.]
   [ 91. 102. 108.]
   [ 94. 103. 105.]]

  [[160. 162. 161.]
   [176. 178. 177.]
   [177. 179. 178.]
   ...
   [ 91.  99. 1

AssertionError: first transform is unvalid

In [8]:
%debug

> /Users/colins/Documents/Master AI/Semester 5/Active Learning Projekt/git/deepaugment/deepaugment/deepaugment/augmenter.py(164)augment_by_policy()
    162         X_portion_aug = transform(hyperparams[i], hyperparams[i+1], X_portion)  # first transform
    163 
--> 164         assert (
    165             X_portion_aug.min() >= -0.1 and X_portion_aug.max() <= 255.1
    166         ), "first transform is unvalid"

array([[[[226.69376 , 226.69376 , 226.69376 ],
         [227.5352  , 227.5352  , 227.5352  ],
         [235.43317 , 235.43317 , 235.43317 ],
         ...,
         [208.89224 , 214.46973 , 217.04395 ],
         [201.12335 , 206.27179 , 209.27502 ],
         [190.96835 , 196.11679 , 198.69101 ]],

        [[224.64238 , 224.64238 , 224.64238 ],
         [228.11453 , 228.11453 , 228.11453 ],
         [237.32614 , 237.32614 , 237.32614 ],
         ...,
         [208.54593 , 212.8363  , 214.98148 ],
         [196.28632 , 200.57666 , 202.72185 ],
         [186.82022 , 191.11058 , 1

In [5]:

from augmenter import transform

transform('gamma-contrast',0.8442657485810175)

NameError: name 'trial_no' is not defined

## 5. See found best policies

In [ ]:
best_policies

,trial_no,A_aug1_type,A_aug1_magnitude,A_aug2_type,A_aug2_magnitude,B_aug1_type,B_aug1_magnitude,B_aug2_type,B_aug2_magnitude,C_aug1_type,...,D_aug1_type,D_aug1_magnitude,D_aug2_type,D_aug2_magnitude,E_aug1_type,E_aug1_magnitude,E_aug2_type,E_aug2_magnitude,mean_late_val_acc,expected_accuracy_increase(%)
0,0,rotate,0.000,rotate,0.000,rotate,0.000,rotate,0.000,rotate,...,rotate,0.000,rotate,0.000,rotate,0.000,rotate,0.00,0.392,0.0
4,2,dropout,0.801,coarse-dropout,0.679,fog,0.582,coarse-dropout,0.759,rotate,...,translate-x,0.135,sharpen,0.150,translate-x,0.386,horizontal-flip,0.45,0.255,-13.7
2,1,gamma-contrast,0.844,coarse-salt-pepper,0.847,brighten,0.384,translate-y,0.057,translate-y,...,emboss,0.836,sharpen,0.648,emboss,0.957,rotate,0.87,0.206,-18.6


## 6. Create Image Generator by found best policies

In [ ]:
import keras
image_gen = deepaug.image_generator_with_top_policies(x_train, keras.utils.to_categorical(y_train))

## 7. Use DeepAugmented images on your model

Let's try  BasicCNN (Child CNN) model which can be created by modules in the package (assuming that your full model is same with BasicCNN). For a proper analysis use WideResNet-28-10, or better, your own model.

In [ ]:
from deepaugment.childcnn import ChildCNN
import logging

cnn_config = {"model":"basicCNN", "logging":logging}
full_model = ChildCNN(input_shape=x_train.shape[1:], num_classes=10, config=cnn_config)

BasicCNN model built as child model.
 Model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)      

In [ ]:
BATCH_SIZE = 64

full_model.model.fit_generator(
    image_gen,
    validation_data=(x_test, keras.utils.to_categorical(y_test)),
    steps_per_epoch=len(x_train) // BATCH_SIZE,
    epochs=5,
    shuffle=True,
    verbose=2
)

Epoch 1/5
Policies are:
[{'A_aug1_type': 'rotate', 'A_aug1_magnitude': 0.0, 'A_aug2_type': 'rotate', 'A_aug2_magnitude': 0.0, 'B_aug1_type': 'rotate', 'B_aug1_magnitude': 0.0, 'B_aug2_type': 'rotate', 'B_aug2_magnitude': 0.0, 'C_aug1_type': 'rotate', 'C_aug1_magnitude': 0.0, 'C_aug2_type': 'rotate', 'C_aug2_magnitude': 0.0, 'D_aug1_type': 'rotate', 'D_aug1_magnitude': 0.0, 'D_aug2_type': 'rotate', 'D_aug2_magnitude': 0.0, 'E_aug1_type': 'rotate', 'E_aug1_magnitude': 0.0, 'E_aug2_type': 'rotate', 'E_aug2_magnitude': 0.0}, {'A_aug1_type': 'dropout', 'A_aug1_magnitude': 0.801, 'A_aug2_type': 'coarse-dropout', 'A_aug2_magnitude': 0.679, 'B_aug1_type': 'fog', 'B_aug1_magnitude': 0.582, 'B_aug2_type': 'coarse-dropout', 'B_aug2_magnitude': 0.759, 'C_aug1_type': 'rotate', 'C_aug1_magnitude': 0.474, 'C_aug2_type': 'shear', 'C_aug2_magnitude': 0.737, 'D_aug1_type': 'translate-x', 'D_aug1_magnitude': 0.135, 'D_aug2_type': 'sharpen', 'D_aug2_magnitude': 0.15, 'E_aug1_type': 'translate-x', 'E_aug1_